1 簡介
嵌入層將正整數（下標）轉換為具有固定大小的向量，如[[4],[20]]->[[0.25,0.1],[0.6,-0.2]]

Embedding層只能作為模型的第一層
```python
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)
``` 
參數
重點關注三個input_dim、output_dim、input_length參數。

- input_dim：大或等於0的整數，字典長度，即輸入數據最大下標+1
- output_dim：大於0的整數，代表全連接嵌入的維度

- embeddings_initializer: 嵌入矩陣的初始化方法，為預定義初始化方法名的字符串，或用於初始化權重的初始化器。
- embeddings_regularizer: 嵌入矩陣的正則項，為Regularizer對像
- embeddings_constraint: 嵌入矩陣的約束項，為Constraints對像
- mask_zero：布爾值，確定是否將輸入中的『0』看作是應該被忽略的『填充』（padding）值，該參數在使用遞歸層處理變長輸入時有用。設置為True的話，模型中後續的層必須都支持masking，否則會拋出異常。如果該值為True，則下標0在字典中不可用，input_dim應設置為|vocabulary| + 1。
- input_length：當輸入序列的長度固定時，該值為其長度。如果要在該層後接Flatten層，然後接Dense層，則必須指定該參數，否則Dense層的輸出維度無法自動推斷。
## 示例一
### 數據準備
創造一個shape為（3，2）的數據，即一共3個數據，每個數據為2維，數據中最大數字為10


In [1]:
import numpy as np
import tensorflow as tf
data = np.array([[0,1],[2,3],[4,10]])
print(data.shape) # (3, 2)
print(data)


(3, 2)
[[ 0  1]
 [ 2  3]
 [ 4 10]]



## 模型搭建與測試
我們將數據轉換為具有固定大小的向量


In [2]:

model_test = tf.keras.models.Sequential()
model_test.add(tf.keras.layers.Embedding(11,2,input_length=2))

model_test.compile('rmsprop', 'mse')
pre = model_test.predict(data)
print(pre)
print(pre.shape)


c:\pywork\ollama\basic5_torch\prj\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
[[[ 0.00401244  0.02178882]
  [-0.0489428  -0.02986949]]

 [[ 0.02268794  0.00041698]
  [-0.01534552  0.01989034]]

 [[-0.03999241  0.04818959]
  [-0.04581785 -0.04527298]]]
(3, 2, 2)


其中Embedding(11,2,input_length=2)

- 第一個參數：因輸入數據中最大數據為10，因此input_dim設置為11，該參數為必填，「input_dim=」省略
- 第二個參數：2，即將每個數字轉換為具有固定大小維度為2的向量，該參數為必填，「output_dim=」省略
- 第三個參數：input_length=2，代表輸入的每個數據的長度，因原數據中每個數據為2維，因此input_length=2

## 查看結果
原數據：[[0,1],[2,3],[4,10]] shape為(3, 2)

經過模型之後：
```
array([[[-0.02706119, -0.03584576],
        [-0.0184832 ,  0.01159693]],

       [[-0.03302763,  0.04805403],
        [-0.04142798, -0.01162308]],

       [[-0.02347708,  0.0004672 ],
        [-0.01176012, -0.04759126]]], dtype=float32)
```        
 
shape為(3, 2, 2)

可以看出原數據中的0變為[-0.02706119, -0.03584576]，1變為[-0.0184832 , 0.01159693]

## 示例二


In [ ]:

# 數據準備
input_array = np.random.randint(1000, size=(32, 10)) # shape為(32,10)
# 構建測試模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
# 模型的輸入size應該為(batch,input_length),並且數據中最大不可超過999
# 模型的輸出shape為(None,10,64),『None』是batch維度
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)
model.summary() 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
(32, 10, 64)


c:\pywork\ollama\basic5_torch\prj\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (32, 10, 64)           │        64,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,000 (250.00 KB)

 Trainable params: 64,000 (250.00 KB)

 Non-trainable params: 0 (0.00 B)

輸出shape
```
(32, 10, 64)
``` 
分析：

經過Embedding層之後，原數據中的0-999這1000個數，都被映射為一個具有64維的向量。維度由2維變為3維。

注意：原數據為32個10維的整數數據，每個數據的範圍為[0,1000) ，正是因為原數據不包含1000，因此Embedding的第一個參數可以設置為1000，如果包含，則需設置為1001

ref

- [overview](https://www.cnblogs.com/bjxqmy/p/13521516.html)
- [embeding](https://coderzcolumn.com/tutorials/artificial-intelligence/keras-word-embeddings-for-text-classification)